# Binary Prediction with a Rainfall Dataset


## Keras - 1 Attempt

## Attempt 1

In [ ]:
# Reproducible binary classification notebook for rainfall prediction
import os, random, time, json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# 2. Load sample submission to infer id_col and target_columns
submission_example = pd.read_csv('sample_submission.csv')
id_col = submission_example.columns[0]
target_columns = submission_example.columns[1:].tolist()

# 3. Load train and test datasets
train_df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# 4. Prepare target encoding
col = target_columns[0]
le = LabelEncoder().fit(train_df[col].astype(str))
y_enc = le.transform(train_df[col].astype(str)).astype(int)

# 5. Drop id and target from features
X = train_df.drop(columns=[id_col] + target_columns, errors='ignore')
id_train = train_df[id_col]
id_test = df_test[id_col]
X_test = df_test.drop(columns=[id_col] + target_columns, errors='ignore')

# 6. Split train & validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y_enc,
    test_size=0.2,
    stratify=y_enc,
    random_state=seed
)

# 7. Drop columns with all missing values
all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train.drop(columns=all_missing, inplace=True)
X_val = X_val[X_train.columns]
X_test = X_test[X_train.columns]

# 8. Identify numeric and categorical features
numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()
# remove high-cardinality cats
cat_cols = [c for c in cat_cols if X_train[c].nunique() < 50]

# 9. Build preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, cat_cols)
])
# Fit and transform
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# 10. Define model architecture based on data size
n_samples, n_features = X_train_proc.shape
if n_samples < 10000 or n_features < 100:
    # smaller dataset: two layers, dropout
    units1 = min(n_features * 2, 128)
    units2 = min(n_features, 64)
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(n_features,)),
        tf.keras.layers.Dense(units1, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(units2, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
else:
    # larger dataset: deeper network
    sizes = [min(int(n_features * i), 1024) for i in (2,1,0.5,0.25)]
    sizes = [s for s in sizes if s >= 16]
    inputs = tf.keras.Input(shape=(n_features,))
    x = inputs
    for s in sizes:
        x = tf.keras.layers.Dense(s, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.4)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)

# 11. Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

# 12. Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# 13. Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_data=(X_val_proc, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# 14. Save results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 15. Prediction & submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
pred_labels = (probs > 0.5).astype(int)
submission = pd.DataFrame({id_col: id_test.values, target_columns[0]: probs})
# output probabilistic prediction for rainfall
submission.to_csv('submission_result.csv', index=False)
print('Notebook run complete.')

2025-07-09 21:22:03.749045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752096123.768278  898713 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752096123.774108  898713 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752096123.790124  898713 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752096123.790139  898713 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752096123.790142  898713 computation_placer.cc:177] computation placer alr

Epoch 1/100


2025-07-09 21:22:06.458931: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


28/28 - 1s - 49ms/step - accuracy: 0.3801 - auc: 0.5648 - loss: 0.9037 - val_accuracy: 0.5091 - val_auc: 0.6990 - val_loss: 0.6978
Epoch 2/100


28/28 - 0s - 5ms/step - accuracy: 0.6273 - auc: 0.6608 - loss: 0.6338 - val_accuracy: 0.7991 - val_auc: 0.8037 - val_loss: 0.5232
Epoch 3/100


...

28/28 - 0s - 4ms/step - accuracy: 0.8647 - auc: 0.8805 - loss: 0.3551 - val_accuracy: 0.8676 - val_auc: 0.8789 - val_loss: 0.3501
Epoch 40/100
28/28 - 0s - 4ms/step - accuracy: 0.8624 - auc: 0.8827 - loss: 0.3555 - val_accuracy: 0.8653 - val_auc: 0.8789 - val_loss: 0.3505
Epoch 41/100
28/28 - 0s - 4ms/step - accuracy: 0.8579 - auc: 0.8865 - loss: 0.3502 - val_accuracy: 0.8676 - val_auc: 0.8786 - val_loss: 0.3506
Epoch 42/100
28/28 - 0s - 5ms/step - accuracy: 0.8647 - auc: 0.8868 - loss: 0.3478 - val_accuracy: 0.8676 - val_auc: 0.8788 - val_loss: 0.3507
Epoch 43/100
28/28 - 0s - 4ms/step - accuracy: 0.8676 - auc: 0.8965 - loss: 0.3359 - val_accuracy: 0.8676 - val_auc: 0.8788 - val_loss: 0.3513
Epoch 44/100
28/28 - 0s - 4ms/step - accuracy: 0.8653 - auc: 0.8889 - loss: 0.3459 - val_accuracy: 0.8676 - val_auc: 0.8789 - val_loss: 0.3508
Epoch 45/100
28/28 - 0s - 4ms/step - accuracy: 0.8659 - auc: 0.8824 - loss: 0.3551 - val_accuracy: 0.8676 - val_auc: 0.8794 - val_loss: 0.3507
Epoch 46/100

28/28 - 0s - 4ms/step - accuracy: 0.8613 - auc: 0.8799 - loss: 0.3607 - val_accuracy: 0.8699 - val_auc: 0.8793 - val_loss: 0.3499
Epoch 47/100


28/28 - 0s - 4ms/step - accuracy: 0.8704 - auc: 0.8895 - loss: 0.3420 - val_accuracy: 0.8676 - val_auc: 0.8791 - val_loss: 0.3495
Epoch 48/100
28/28 - 0s - 4ms/step - accuracy: 0.8607 - auc: 0.8813 - loss: 0.3551 - val_accuracy: 0.8676 - val_auc: 0.8795 - val_loss: 0.3496
Epoch 49/100


28/28 - 0s - 4ms/step - accuracy: 0.8636 - auc: 0.8883 - loss: 0.3473 - val_accuracy: 0.8676 - val_auc: 0.8794 - val_loss: 0.3493
Epoch 50/100
28/28 - 0s - 4ms/step - accuracy: 0.8636 - auc: 0.8895 - loss: 0.3430 - val_accuracy: 0.8676 - val_auc: 0.8790 - val_loss: 0.3497
Epoch 51/100
28/28 - 0s - 4ms/step - accuracy: 0.8704 - auc: 0.8894 - loss: 0.3446 - val_accuracy: 0.8676 - val_auc: 0.8800 - val_loss: 0.3500
Epoch 52/100


28/28 - 0s - 4ms/step - accuracy: 0.8664 - auc: 0.8823 - loss: 0.3532 - val_accuracy: 0.8699 - val_auc: 0.8797 - val_loss: 0.3490
Epoch 53/100
28/28 - 0s - 4ms/step - accuracy: 0.8630 - auc: 0.8857 - loss: 0.3491 - val_accuracy: 0.8676 - val_auc: 0.8793 - val_loss: 0.3501
Epoch 54/100
28/28 - 0s - 4ms/step - accuracy: 0.8642 - auc: 0.8882 - loss: 0.3482 - val_accuracy: 0.8676 - val_auc: 0.8791 - val_loss: 0.3506
Epoch 55/100
28/28 - 0s - 4ms/step - accuracy: 0.8710 - auc: 0.8926 - loss: 0.3393 - val_accuracy: 0.8676 - val_auc: 0.8793 - val_loss: 0.3497
Epoch 56/100
28/28 - 0s - 4ms/step - accuracy: 0.8636 - auc: 0.8885 - loss: 0.3461 - val_accuracy: 0.8676 - val_auc: 0.8789 - val_loss: 0.3503
Epoch 57/100
28/28 - 0s - 4ms/step - accuracy: 0.8676 - auc: 0.8882 - loss: 0.3437 - val_accuracy: 0.8653 - val_auc: 0.8790 - val_loss: 0.3513
Epoch 58/100
28/28 - 0s - 4ms/step - accuracy: 0.8642 - auc: 0.8916 - loss: 0.3428 - val_accuracy: 0.8630 - val_auc: 0.8780 - val_loss: 0.3515
Epoch 59/100

## Keras Tuner - 1 Attempt

# Attempt 1 

In [ ]:
import os, random, time, json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# 1. Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# 2. Load sample submission to infer id_col and target_columns
submission_example = pd.read_csv('sample_submission.csv')
id_col = submission_example.columns[0]
target_columns = submission_example.columns[1:].tolist()

# 3. Load train and test datasets
train_df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# 4. Prepare target encoding
col = target_columns[0]
le = LabelEncoder().fit(train_df[col].astype(str))
y_enc = le.transform(train_df[col].astype(str)).astype(int)

# 5. Drop id and target from features
X = train_df.drop(columns=[id_col] + target_columns, errors='ignore')
id_train = train_df[id_col]
id_test = df_test[id_col]
X_test = df_test.drop(columns=[id_col] + target_columns, errors='ignore')

# 6. Split train & validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y_enc,
    test_size=0.2,
    stratify=y_enc,
    random_state=seed
)

# 7. Drop columns with all missing values
all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train.drop(columns=all_missing, inplace=True)
X_val = X_val[X_train.columns]
X_test = X_test[X_train.columns]

# 8. Identify numeric and categorical features
numeric_cols = X_train.select_dtypes(include=['number']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object','category']).columns.tolist()
# remove high-cardinality cats
cat_cols = [c for c in cat_cols if X_train[c].nunique() < 50]

# 9. Build preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, cat_cols)
])
# Fit and transform
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# 10. Define model architecture using Keras Tuner
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

n_features = X_train_proc.shape[1]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, 64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = BatchNormalization()(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        return model

# Initialize the Bayesian tuner
bs = 32  # batch size
ep = 20   # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

start_time = time.time()  # Start timing

# Retrain the model with the original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
duration = time.time() - start_time  # Calculate duration


# 13. Save results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# 14. Prediction & submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
pred_labels = (probs > 0.5).astype(int)
submission = pd.DataFrame({id_col: id_test.values, target_columns[0]: probs})
# output probabilistic prediction for rainfall
submission.to_csv('submission_result.csv', index=False)
print('Notebook run complete.')

Trial 10 Complete [00h 00m 20s]
val_loss: 0.3696853220462799

Best val_loss So Far: 0.3444618880748749
Total elapsed time: 00h 03m 36s
Epoch 1/100


55/55 - 14s - 251ms/step - accuracy: 0.6901 - loss: 0.6443 - val_accuracy: 0.7534 - val_loss: 0.5476
Epoch 2/100


55/55 - 1s - 11ms/step - accuracy: 0.8151 - loss: 0.4624 - val_accuracy: 0.7740 - val_loss: 0.5195
Epoch 3/100


55/55 - 1s - 12ms/step - accuracy: 0.8293 - loss: 0.4433 - val_accuracy: 0.8379 - val_loss: 0.4864
Epoch 4/100


55/55 - 1s - 12ms/step - accuracy: 0.8408 - loss: 0.4054 - val_accuracy: 0.8082 - val_loss: 0.4044
Epoch 5/100


55/55 - 1s - 12ms/step - accuracy: 0.8311 - loss: 0.4087 - val_accuracy: 0.8425 - val_loss: 0.3843
Epoch 6/100


55/55 - 1s - 12ms/step - accuracy: 0.8356 - loss: 0.4105 - val_accuracy: 0.8767 - val_loss: 0.3728
Epoch 7/100


55/55 - 1s - 12ms/step - accuracy: 0.8465 - loss: 0.3666 - val_accuracy: 0.8607 - val_loss: 0.3632
Epoch 8/100
55/55 - 1s - 9ms/step - accuracy: 0.8482 - loss: 0.3764 - val_accuracy: 0.8699 - val_loss: 0.3681
Epoch 9/100


55/55 - 1s - 11ms/step - accuracy: 0.8493 - loss: 0.3618 - val_accuracy: 0.8767 - val_loss: 0.3485
Epoch 10/100
55/55 - 1s - 9ms/step - accuracy: 0.8579 - loss: 0.3610 - val_accuracy: 0.8721 - val_loss: 0.3546
Epoch 11/100
55/55 - 0s - 9ms/step - accuracy: 0.8602 - loss: 0.3553 - val_accuracy: 0.8539 - val_loss: 0.3696
Epoch 12/100
55/55 - 0s - 8ms/step - accuracy: 0.8659 - loss: 0.3601 - val_accuracy: 0.8630 - val_loss: 0.3762
Epoch 13/100
55/55 - 1s - 9ms/step - accuracy: 0.8647 - loss: 0.3472 - val_accuracy: 0.8607 - val_loss: 0.3722
Epoch 14/100
55/55 - 0s - 9ms/step - accuracy: 0.8516 - loss: 0.3633 - val_accuracy: 0.8744 - val_loss: 0.3686
Epoch 15/100
55/55 - 1s - 9ms/step - accuracy: 0.8579 - loss: 0.3347 - val_accuracy: 0.8699 - val_loss: 0.3673
Epoch 16/100
55/55 - 0s - 8ms/step - accuracy: 0.8556 - loss: 0.3462 - val_accuracy: 0.8584 - val_loss: 0.3697
Epoch 17/100
55/55 - 0s - 9ms/step - accuracy: 0.8659 - loss: 0.3423 - val_accuracy: 0.8630 - val_loss: 0.3747
Epoch 18/100


In [1]:
print(duration)

26.242344
